In [1]:
siteUrl = "http://www.ksl.com/classifieds/search/"
siteUrl = "?".join([siteUrl, "keyword="])
siteUrl = "&".join([siteUrl, "category%5B%5D=Recreational+Vehicles"])
siteUrl = "&".join([siteUrl, "subCategory%5B%5D=Travel+Trailers%2C+5th+Wheel"])
siteUrl = "&".join([siteUrl, "priceFrom=10000"])
siteUrl = "&".join([siteUrl, "priceTo=100000000"])
siteUrl = "&".join([siteUrl, "sellerType%5B%5D=Private"])
siteUrl = "&".join([siteUrl, "city="])
siteUrl = "&".join([siteUrl, "state="])
siteUrl = "&".join([siteUrl, "zip="])
siteUrl = "&".join([siteUrl, "miles=25"])
siteUrl = "&".join([siteUrl, "sort=0"])
print(siteUrl)

http://www.ksl.com/classifieds/search/?keyword=&category%5B%5D=Recreational+Vehicles&subCategory%5B%5D=Travel+Trailers%2C+5th+Wheel&priceFrom=10000&priceTo=100000000&sellerType%5B%5D=Private&city=&state=&zip=&miles=25&sort=0


In [2]:
from selenium import webdriver
from base64 import b64encode

proxy = {'host': '23.19.51.245', 'port': '40791', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>'}
# Retrieve the page with tag results and set it up to be scraped
fp = webdriver.FirefoxProfile()
fp.set_preference('network.proxy.type', 1)
fp.set_preference('network.proxy.http', proxy['host'])
fp.set_preference('network.proxy.http_port', int(proxy['port']))
fp.set_preference('network.proxy.no_proxies_on', 'localhost, 127.0.0.1')
fp.add_extension('closeproxy.xpi')
credentials = '{usr}:{pwd}'.format(**proxy)
credentials = b64encode(credentials.encode('ascii')).decode('utf-8')
fp.set_preference('extensions.closeproxyauth.authtoken', credentials)

#set some privacy settings
fp.set_preference( "places.history.enabled", False )
fp.set_preference( "privacy.clearOnShutdown.offlineApps", True )
fp.set_preference( "privacy.clearOnShutdown.passwords", True )
fp.set_preference( "privacy.clearOnShutdown.siteSettings", True )
fp.set_preference( "privacy.sanitize.sanitizeOnShutdown", True )
fp.set_preference( "signon.rememberSignons", False )
fp.set_preference( "network.cookie.lifetimePolicy", 2 )
fp.set_preference( "network.dns.disablePrefetch", True )
fp.set_preference( "network.http.sendRefererHeader", 0 )

#if you're really hardcore about your security
#js can be used to reveal your true i.p.
fp.set_preference( "javascript.enabled", False )

#get a huge speed increase by not downloading images
fp.set_preference( "permissions.default.image", 2 )

ffDriver = webdriver.Firefox(fp)

In [3]:
import time

# set timeout information
ffDriver.set_page_load_timeout(5)

finished = 0
while finished == 0:
    try:
        ffDriver.get(siteUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        time.sleep(5)

Message: Error loading page, timed out (checkLoad)

Message: Error loading page, timed out (checkLoad)



In [4]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

def wait_for(condition_function):
    start_time = time.time()
    while time.time() < start_time + 6:
        if condition_function():
            return True
        else:
            time.sleep(0.1)
    raise Exception(
        'Timeout waiting for {}'.format(condition_function.__name__)
    )

class wait_for_page_load(object):

    def __init__(self, browser):
        self.ffDriver = ffDriver

    def __enter__(self):
        self.old_page = self.ffDriver.find_element_by_tag_name('html')

    def page_has_loaded(self):
        new_page = self.ffDriver.find_element_by_tag_name('html')
        time.sleep(0.1)
        return new_page.id != self.old_page.id

    def __exit__(self, *_):
        wait_for(self.page_has_loaded)

def get_page_urls(driver):
    page_source = driver.page_source
    current_url = driver.current_url
    
    # Retrieve the page with tag results and set it up to be scraped
    soup = BeautifulSoup(page_source, 'lxml')

    pageLinks = soup.select('h2 a.link')
    for pageLink in pageLinks:
        pageUrl = urllib.parse.urljoin(current_url, re.sub(' ', '%20', pageLink['href']))
        if pageUrl not in seen_urls_array:
            seen_urls_array.append(pageUrl)

In [5]:
from bs4 import BeautifulSoup
import urllib.parse
import re

seen_urls_array = []
get_page_urls(ffDriver)
while True:
    try:
        with wait_for_page_load(ffDriver):
            ffDriver.find_element_by_link_text('Next »').click()
        get_page_urls(ffDriver)
    except NoSuchElementException as e:
        print('The "Next »" link is missing: ' + str(e))
        break  # cannot click the link anymore
    except TimeoutException as e:
        print('We timed out: ' + str(e))
        break  # cannot click the link anymore
    except Exception as e:
        if re.search(r"Timeout", str(e)):
            break # cannot click the link any more
        else:
            print(str(e))

The "Next »" link is missing: Message: Unable to locate element: html



In [7]:
ffDriver.quit()

In [8]:
from random import randint
import requests

pageUrl = seen_urls_array[randint(0,len(seen_urls_array)-1)]
print(pageUrl)

page = requests.get(url=pageUrl)
soup = BeautifulSoup(page.content, 'lxml')

http://www.ksl.com/classifieds/listing/42977318


In [9]:
model = soup.select("div.listing-details.details-element > h2")
if(len(model)):
    model = model[0].get_text().strip()
else:
    model = "Unknown"
print(model)

price = soup.select("div.listing-details.details-element > h3")
if(len(price)):
    price = price[0].get_text().strip()
else:
    price = "Unknown"
print(price)

phone = soup.select('li.contact-homePhone > p > span')
if(len(phone)>=2):
    phone = phone[1].get_text().strip()
else:
    phone = ""
print(phone)

Cherokee
$20,000.00
(435) 830-7675


In [28]:
import sys

n = 0
for pageUrl in seen_urls_array:
    #print(pageUrl)
    page = requests.get(url=pageUrl)
    soup = BeautifulSoup(page.content, 'lxml')
    model = soup.select("div.listing-details.details-element > h2")
    if(len(model)):
        model = model[0].get_text().strip()
    else:
        model = "Unknown"
    price = soup.select("div.listing-details.details-element > h3")
    if(len(price)):
        price = price[0].get_text().strip()
    else:
        price = "Unknown"
    phone = soup.select('li.contact-homePhone > p > span')
    if(len(phone)>=2):
        phone = phone[1].get_text().strip()
    else:
        phone = ""
    if len(phone):
        bigFile = open('kslTrailers.txt', 'a', encoding='utf-8')
        byteString = model + '\t' + price + '\t' + phone + '\n'
        try:
            bigFile.write(byteString)
        except Exception as e:
            print(str(e))
            print(model + '\t' + price + '\t' + phone + '\n')
        n = n + 1
        bigFile.close()
print(str(n) + " records written")

193 records written
